In [ ]:
import covidcountydata as ccd
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

## Covid County Data

[Covid County Data](https://covidcountydata.org/) is an organization that works to collect county-level COVID-19 relevant data.

The main component of their Python client library is the `Client` object which can query their database and retrieve the data that you're interested in

### Client

In [ ]:
c = ccd.Client()

You can register for an API key by using the `register` method. The first time your emai is used, an API key will be generated and saved onto your computer.

In [ ]:
# change to enter your email address!!!
c.register("spencer.lyon@valorumdata.com");

You can see the names of each dataset by looking at the `datasets` property

In [ ]:
c.datasets

### Making a request

We will select all of the demographic variables and a subset of variables from the `covid_us` endpoint

In [ ]:
covid_variables = [
    "cases_total",
    "deaths_total",
    "tests_total",
]

c.demographics(
    state=12
).covid_us(
    variable=covid_variables
)

### Fetch

Once we've specified what we want our request to entail, we can make the request

In [ ]:
df = c.fetch()

In [ ]:
df.info()

The three COVID variables are all cumulative counts.

Let's compute the daily new values for each of these variables, as well as a 7 day moving avergae 

In [ ]:
# Make sure things are sorted by location and date
df = df.sort_values(["location", "dt"])

for var in ["cases", "deaths", "tests"]:
    # Once they're sorted we can compute the new cases/deaths/tests
    # by taking the one element difference
    df[f"{var}_new"]= df.groupby("location")[f"{var}_total"].diff()
    
    # Get moving averages of the new variables given the daily variation
    # driven by day-of-week and other outside factors
    df[f"{var}_7dma"] = df.groupby("location")[f"{var}_new"].apply(
        lambda x: x.rolling(window=7, min_periods=7).mean()
    )

Let's visualize this data for Florida as a whole

In [ ]:
# get florida data since start of march
fl_df = df.query("(location == 12) & (dt >= '2020-03-01')")

fig, ax = plt.subplots(2, figsize=(12, 8))

fig.suptitle("Florida COVID-19 cases and deaths")

# Plot cases
fl_df.plot(
    x="dt", y="cases_new", kind="line", linewidth=1.0, 
    color="k", alpha=0.5,
    legend=False, ax=ax[0]
)
fl_df.plot(
    x="dt", y="cases_7dma", kind="line", linewidth=2.0,
    color="k", alpha=1.0,
    legend=False, ax=ax[0]
)

# Get labels just right
xmin, xmax = ax[0].get_xlim()
ax[0].annotate(
    "7 day moving average",
    xy=(xmin + 0.70*(xmax-xmin), 1500),
    color="k"
)
ax[0].annotate(
    "Raw values",
    xy=(xmin + 0.6*(xmax-xmin), 5000),
    color="k", alpha=0.5
)
ax[0].annotate(
    "Case data",
    xy=(xmin + 0.025*(xmax-xmin), 14000),
    color="k", alpha=1.0, fontsize=18,
)
ax[0].set_xlabel("")


# plot deaths
fl_df.plot(
    x="dt", y="deaths_new", kind="line", linewidth=1.0,
    color="k", alpha=0.5,
    legend=False, ax=ax[1]
)
fl_df.plot(
    x="dt", y="deaths_7dma", kind="line", linewidth=2.0,
    color="k", alpha=1.0,
    legend=False, ax=ax[1]
)

# get labels just right
xmin, xmax = ax[1].get_xlim()
ax[1].annotate(
    "7 day moving average",
    xy=(xmin + 0.70*(xmax-xmin), 0),
    color="k"
)
ax[1].annotate(
    "Raw values",
    xy=(xmin + 0.6*(xmax-xmin), 65),
    color="k", alpha=0.5
)
ax[1].annotate(
    "Death data",
    xy=(xmin + 0.025*(xmax-xmin), 240),
    color="k", alpha=1.0, fontsize=18,
)
ax[1].set_xlabel("Date")

for _ax in ax:
    _ax.spines["top"].set_visible(False)
    _ax.spines["right"].set_visible(False)
    

In [ ]:
fig.savefig("example.png")

## Interactive Graphing with `ipywidgets`

Let's take the graph we just made and make it more interactive using the `ipywidget` tools

In [ ]:
from ipywidgets import (
    interactive_output, Checkbox, Dropdown, IntSlider,
    Box, HBox, VBox
)

In [ ]:
def get_data(state):

    c = ccd.Client()

    covid_variables = [
        "cases_total",
        "deaths_total",
        "tests_total",
    ]

    c.demographics(
        state=state, variable=["Total population"]
    ).covid_us(
        variable=covid_variables
    )
    
    df = c.fetch()

    # Make sure things are sorted by location and date
    df = df.sort_values(["location", "dt"])

    # Once they're sorted we can compute the new cases/deaths/tests
    # by taking the one element difference
    df["cases_new"] = df.groupby("location")["cases_total"].diff()
    df["deaths_new"] = df.groupby("location")["deaths_total"].diff()
    df["tests_new"] = df.groupby("location")["tests_total"].diff()

    return df


In [ ]:
df = get_data(12)

In [ ]:
def scale_data(df, pp=100_000):
    scale_vars = [
        "cases_total",
        "cases_new",
        "deaths_total",
        "deaths_new",
        "tests_total",
        "tests_new"
    ]
    
    for _var in scale_vars:
        df.loc[:, _var] = df.loc[:, _var] / (df["Total population"] / pp)
    
    return df

In [ ]:
def compute_rolling_means(df, window=7):
    # Get moving averages of the new variables given the daily variation
    # driven by day-of-week and other outside factors
    df = df.assign(
        cases_ma=df.groupby("location")["cases_new"].apply(
            lambda x: x.rolling(window=window, min_periods=window).mean()
        ),
        deaths_ma=df.groupby("location")["deaths_new"].apply(
            lambda x: x.rolling(window=window, min_periods=window).mean()
        ),
        tests_ma=df.groupby("location")["tests_new"].apply(
            lambda x: x.rolling(window=window, min_periods=window).mean()
        )
    )

    return df

In [ ]:
def fill_axis(ax, df, var, scale, window):
    df.plot(
        x="dt", y=f"{var.lower()}_new", kind="line",
        linewidth=1.0, color="k", alpha=0.5,
        legend=False, ax=ax
    )
    df.plot(
        x="dt", y=f"{var.lower()}_ma", kind="line", linewidth=2.0,
        color="k", alpha=1.0,
        legend=False, ax=ax
    )

    xmin, xmax = ax.get_xlim()
    ymin, ymax = ax.get_ylim()
    ax.annotate(
        f"{window} day moving average",
        xy=(xmin + 0.20*(xmax-xmin), ymin + 0.4*(ymax-ymin)),
        color="k"
    )
    ax.annotate(
        "Raw values",
        xy=(xmin + 0.6*(xmax-xmin), ymin + 0.7*(ymax-ymin)),
        color="k", alpha=0.5
    )
    ax.annotate(
        f"{var} data" + ("" if not scale else " (per 100,000 population)"),
        xy=(xmin + 0.025*(xmax-xmin), 0.9*ymax),
        color="k", alpha=1.0, fontsize=18,
    )

    return ax

In [ ]:
def generate_graph(df, fips, var1, var2, scaled=False, window=7):

    # Restrict to the one location
    df = df.query(f"(location == {fips}) & (dt >='2020-03-01')")

    # Do we want to scale the values as instances per 100,000 people
    if scaled:
        df = scale_data(df, 100_000)
    
    # Compute the rolling means
    df = compute_rolling_means(df, window)

    # Create the figure
    fig, ax = plt.subplots(2, figsize=(12, 8))
    fig.suptitle(f"Florida COVID-19 {var1} and {var2}")

    fill_axis(ax[0], df, var1, scaled, window)
    fill_axis(ax[1], df, var2, scaled, window)
    ax[0].set_xlabel("")
    ax[1].set_xlabel("Date")

    for _ax in ax:
        _ax.spines["top"].set_visible(False)
        _ax.spines["right"].set_visible(False)

In [ ]:
# Create the interactive buttons
scaled = Checkbox(
    value=False,
    description='Report population scaled values',
)

fips = Dropdown(
    options=df["location"].unique(),
    value=df["location"].min()
)
var1 = Dropdown(
    options=['Cases', 'Deaths', 'Tests'],
    value='Cases'
)

var2 = Dropdown(
    options=['Cases', 'Deaths', 'Tests'],
    value='Deaths'
)


window = IntSlider(
    value=7,
    min=3,
    max=14,
    step=1,
    description='MA window:',
)

ui= VBox(
    [
        HBox([fips, var1, var2]),
        HBox([scaled, window]),
    ]
)


In [ ]:
def f(fips, var1, var2, scaled, window):
    display(generate_graph(df, fips, var1, var2, scaled, window))

out = interactive_output(
    f, {
        "fips": fips,
        "var1": var1,
        "var2": var2,
        "scaled": scaled,
        "window": window
    }
)

display(ui, out);